In [ ]:
# 카테고리 ID 매칭
# 카테고리 1개에서 상품 정보 400개 가져오기

In [165]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import xlrd
import tqdm

In [9]:
# 상품 카테고리 정보

df = pd.read_excel('../data/category_20230518_132842.xls', engine='xlrd')
df['url'] = df['카테고리번호'].apply(lambda x: 'https://search.shopping.naver.com/search/category?catId=' + str(x))
df.head(10)

,카테고리번호,대분류,중분류,소분류,세분류
0,50003307,가구/인테리어,DIY자재/용품,가구부속품,가구다리
1,50003308,가구/인테리어,DIY자재/용품,가구부속품,가구바퀴
2,50003309,가구/인테리어,DIY자재/용품,가구부속품,경첩/꺽쇠/자석철물류
3,50003314,가구/인테리어,DIY자재/용품,가구부속품,기타가구부속품
4,50003310,가구/인테리어,DIY자재/용품,가구부속품,나사/못
5,50003311,가구/인테리어,DIY자재/용품,가구부속품,로프/철망
6,50003312,가구/인테리어,DIY자재/용품,가구부속품,서랍레일
7,50003313,가구/인테리어,DIY자재/용품,가구부속품,소켓/옷걸이봉
8,50001066,가구/인테리어,DIY자재/용품,데코스티커,NaN
9,50008228,가구/인테리어,DIY자재/용품,리모델링,싱크대


In [344]:
# 크롤링 결과 저장할 데이터프레임 생성
crawl_df = pd.DataFrame(columns=['title', 'image_url', 'price', 'mall_price_list', 'brand', 
                                 'cat1', 'cat2', 'cat3', 'cat4'])
crawl_df.to_csv('../data/crawl_df.csv', encoding='utf-8-sig', index=False)

In [389]:
driver = webdriver.Chrome('chromedriver')

for index, row in df.iterrows():
    
    cat1 = row['대분류']
    cat2 = row['중분류']
    cat3 = row['소분류']
    cat4 = row['세분류']
    
    # 카테고리 url
    cat_url = row['url']
    
    # 10페이지까지 접속
    for i in range(1, 11):
        url = str(cat_url) + '&pagingIndex={}&pagingSize=40'.format(i)
        
        try:
            # url 접속 (1개 카테고리 내 1개 페이지)
            driver.get(url)
            
            # 가격비교
            driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/ul/li[2]/a').click()

            # 40개 상품이 모두 노출될 때까지 스크롤
            before_h=driver.execute_script("return window.scrollY")

            while True :
                driver.find_element(by=By.CSS_SELECTOR, value="body").send_keys(Keys.END) #맨 아래로 스크롤 내린다
                time.sleep(2) #스크롤 하는 동안의 로딩시간

                after_h=driver.execute_script("return window.scrollY")

                if after_h==before_h:
                    break
                before_h=after_h
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # 페이지 내 상품 url 리스트
            item_list = soup.find_all(class_="basicList_title__VfX3c")
            item_url_list = [item.find('a')['href'] for item in item_list]
            
            for item_url in item_url_list:
                response = requests.get(item_url)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                
                # 상품명
                title = soup.find(class_="top_summary_title__ViyrM").find('h2').text
                
                # 상품 이미지 url
                image_url = soup.find(class_="image_thumb__IB9B3").find('img')['src']
                
                # 상품 가격
                price = soup.find(class_="lowestPrice_num__A5gM9").text
                
                # 판매처별 최저가
                mall_list = []
                for i in soup.find_all(class_="productList_mall__JtWmC")[1:]:
                    if i.text and i.text != '네이버페이' :
                        mall_list.append(i.find('span').text)
                    else:
                        mall_list.append(i.find('img')['alt'])

                price_list = []
                for i in soup.find_all(class_="productList_price__2eGt4")[1:]:
                    price_list.append(i.find('span').text)

                mall_price_list = tuple(zip(mall_list, price_list))
                
                # 브랜드
                brand = soup.find(class_="lowestPrice_mall__qM0AT").text
                
                new_df = pd.DataFrame(data={
                            'title':[title],
                            'image_url':[image_url],
                            'price':[price],
                            'mall_price':[mall_price],
                            'brand':[brand],
                            'cat1':[cat1],
                            'cat2':[cat2],
                            'cat3':[cat3],
                            'cat4':[cat4]})

                new_df.to_csv('../data/crawl_df.csv', mode='a', index=False, header=False, encoding='utf-8-sig')
            
        except:
            pass